# Methane detection hackathon

In [166]:
# [Hachem] pip install all necessary packages
# [Hachem] Here it is commented for python environment reasons
# %pip install pandas rasterio

### 1. Import statements

In [167]:
# Import all used libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import rasterio
from rasterio.plot import show, show_hist
import warnings
warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)

In [168]:
# [Hachem] : Google collab specific charging of files. Necessary to be able to read files in google collab

# from google.colab import drive
# drive.mount('/content/drive')
PREPENDING_PATH = "AugmentData/" # [Hachem] : Different for Google Drive, otherwise adapted.


In [169]:
print("Number of images : "+str(len(pd.read_csv(PREPENDING_PATH + "metadata_aug.csv"))))

Number of images : 4300


### 2. Metadata and Data loading

In [170]:
# [Hachem] Load metadata
METADATA = pd.read_csv(PREPENDING_PATH +  "/metadata_aug.csv")

In [171]:
# Show the first 5 elements of METADATA.
METADATA.head(5)

,date,id_coord,plume,set,lat,lon,coord_x,coord_y,path
0,20230223,id_6675,yes,train,31.528750,74.330625,24,47,images/plume/20230223_methane_mixing_ratio_id_...
1,20230103,id_2542,yes,train,35.538000,112.524000,42,37,images/plume/20230103_methane_mixing_ratio_id_...
2,20230301,id_6546,yes,train,21.060000,84.936667,58,15,images/plume/20230301_methane_mixing_ratio_id_...
3,20230225,id_6084,yes,train,26.756667,80.973333,28,62,images/plume/20230225_methane_mixing_ratio_id_...
4,20230105,id_2012,yes,train,34.800000,40.770000,59,44,images/plume/20230105_methane_mixing_ratio_id_...


In [172]:
# Load GPU - hopefully ?
# Set the device
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [173]:
# [Hachem] Charger tout les fichiers
plume_count = 0 # Total number of plume images
no_plume_count = 0 # Total number of no plume images

plume_arrays = [] # Store numpy representation of original plume images
plume_positions = [] # Store positions of original plumes
plume_arrays_ids = [] # Store the unique identifiers of each original plume
plume_arrays_aug_imp = [] # Store numpy representation of augmented plume images
plume_positions_aug_imp = [] # Store positions of augmented plumes
plume_arrays_aug_imp_ids = [] # Store the unique identifiers of each augmented plume

no_plume_arrays = [] # Store numpy representation of original non plume images
no_plume_arrays_ids = [] # Store the unique identifiers of each original non plume
no_plume_arrays_aug_imp = [] # Store numpy representation of augmented non plume images
no_plume_arrays_aug_imp_ids = [] # Store the unique identifiers of each augmented non plume

# [Hachem] Load all images.
for index, row in METADATA.iterrows():
    # [Hachem] String preprocessing
    string_p = row.path.replace("\\", "/")
    string_p = string_p.replace("train/", "")
    string_p = string_p.replace(".tif", "")

    # [Hachem] Load image
    img = rasterio.open(PREPENDING_PATH + string_p + ".tif")

    # [Hachem] Load data into appropriate arrays
    if (row.set == "train"): # or row.set == "augmented_perfect"):
        if (row.plume == "yes"):
            plume_arrays.append(img.read(1))
            plume_positions.append([row.coord_x/64, row.coord_y/64])
            plume_arrays_ids.append(str(row.date) + row.id_coord)
            
            plume_count += 1
        elif (row.plume == "no"):
            no_plume_arrays.append(img.read(1))
            no_plume_arrays_ids.append(str(row.date) + row.id_coord)
            
            no_plume_count += 1
    else:
        if (row.plume == "yes"):
            plume_arrays_aug_imp.append(img.read(1))
            plume_positions_aug_imp.append([row.coord_x/64, row.coord_y/64])
            plume_arrays_aug_imp_ids.append(str(row.date) + row.id_coord)
            
            plume_count += 1
        elif (row.plume == "no"):
            no_plume_arrays_aug_imp.append(img.read(1))
            no_plume_arrays_aug_imp_ids.append(str(row.date) + row.id_coord)
            
            no_plume_count += 1
    
    img.close()

In [174]:
print("Total number of plumes : ", plume_count)
print("Total number of non_plume : ", no_plume_count)

Total number of plumes :  2160
Total number of non_plume :  2140


# 3. Set up Classes

In [175]:
# [Hachem] Preprocessing tools
# Transform for set grayscale data to [0.0, 1.0], and then normalize it around median value 0.5
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))]) # Only one value because gray-scale

In [176]:
# [Hachem] DataSet Loader
# This is to be used in conjunction with Pytorch's torch.utils.data.DataLoader.
# DataSet Loader for PlumeNoPlume classifier

class PlumeNoPlumeDataSet(torch.utils.data.Dataset):
    """
    Pytorch DataSet for PlumeNoPlume DataSet
    Arguments :
        - images_numpy : a numpy array of all the 3-dimensional numpy representation of images
        - labels : all of the corresponding labels
        - transform : pytorch transform, for normalizing data.
    """
    def __init__(self, images_numpy, labels, transform=None):
        self.data = images_numpy
        self.labels = labels
        self.transform = transform

    # Self-explanatory
    def __len__(self):
        return len(self.data)

    # Does the preprocessing of the image and returns numpy arrays of the data.
    def __getitem__(self, idx):
        image = np.asarray(self.data[idx])
        label = np.asarray(self.labels[idx])
        if self.transform:
            image = self.transform(image)
        return image, label

# DataSet Loader for Plume position network
class PlumePositionDataSet(torch.utils.data.Dataset):
    """
    Pytorch DataSet for PlumePosition DataSet
    Arguments :
        - images_numpy : same as PlumeNoPlumeDataSet
        - positions : 2 D array with positions between 0 and 1 for of x then of y
        - transform : same as PlumeNoPlumeDataSet
    """
    def __init__(self, images_numpy, positions, transform=None):
        self.data = images_numpy
        self.positions = positions
        self.transform = transform

    # Self-explanatory
    def __len__(self):
        return len(self.data)

    # Does the preprocessing of the image and returns numpy arrays of the data.
    def __getitem__(self, idx):
        image = np.asarray(self.data[idx])
        positions = np.asarray(self.positions[idx])
        if self.transform:
            image = self.transform(image)
        return image, positions

# [Hachem] : Prepares the training and validation sets with approriate ratio.
def prepare_plume_noplume_train_validation(plume, plume_ids, no_plume, no_plume_ids, plume_aug, plume_aug_ids, no_plume_aug, no_plume_aug_ids, ratio):
    """
    Splits data into train and validation
    Inputs :
        - plume : Plume images
        - plume_ids : Unique Identifiers for Plume images
        - no_plume : No Plume images
        - no_plume_ids : Unique Identifiers for No Plume images
        - plume_aug : Augmented Plume images
        - plume_aug_ids : Unique Identifiers for Augmented Plume images
        - no_plume_aug : Augmented No Plume images
        - no_plume_aug_ids : Unique Identifiers for Augmented No Plume images
        - ratio : probability to add to training instead of validation.
    Returns :
        - images_numpy_train, labels_train, images_numpy_validation, labels_validation
    """
    images_numpy_train, labels_train, images_numpy_validation, labels_validation = [], [], [], []

    # The following arrays are to make sure there is no augmented data of a validation-set image, in training set.
    validation_plume_ids = []
    validation_no_plume_ids = [] 
    
    for i in range(len(no_plume)):
        if (np.random.random() < ratio):
            images_numpy_train.append(no_plume[i])
            labels_train.append(0)
        else:
            images_numpy_validation.append(no_plume[i])
            labels_validation.append(0)
            validation_no_plume_ids.append(no_plume_ids[i])
    
    for i in range(len(plume)):
        if (np.random.random() < ratio):
            images_numpy_train.append(plume[i])
            labels_train.append(1);
        else:
            images_numpy_validation.append(plume[i])
            labels_validation.append(1);
            validation_plume_ids.append(plume_ids[i])
    
    
    for i in range(len(no_plume_aug)):
        if (not no_plume_aug_ids[i] in validation_no_plume_ids):
            images_numpy_train.append(no_plume_aug[i])
            labels_train.append(0);

    for i in range(len(plume_aug)):
        if (not plume_aug_ids[i] in validation_plume_ids):
            images_numpy_train.append(plume_aug[i])
            labels_train.append(1);
    
    return np.array(images_numpy_train, dtype=np.float32), np.array(labels_train, dtype=np.float32), np.array(images_numpy_validation, dtype=np.float32), np.array(labels_validation, dtype=np.float32)

def prepare_plume_positions_train_validation(plume, plume_ids, positions_plume, plume_aug, plume_aug_ids, positions_plume_aug, ratio):
    """
    Prepares training and validation sets.
    Input :
        - plume : Plume images
        - plume_ids : Identifiers of plume images
        - positions_plume : Positions of plume images
        - plume_aug : Augmented plume images
        - plume_aug_ids : Identifiers of augmented plume images
        - ratio : probabilistic split ratio.
    Outputs :
        - images_numpy_train, labels_train, images_numpy_validation, labels_validation
    """
    images_numpy_train, positions_train, images_numpy_validation, positions_validation = [], [], [], []
    
    validation_plume_ids = []
    
    for i in range(len(plume)):
        if (np.random.random() < ratio):
            images_numpy_train.append(plume[i])
            positions_train.append(positions_plume[i]);
        else:
          images_numpy_validation.append(plume[i])
          positions_validation.append(positions_plume[i]);
          validation_plume_ids.append(plume_ids[i])
    
    for i in range(len(plume_aug)):
        if (not plume_aug_ids[i] in validation_plume_ids):
            images_numpy_train.append(plume_aug[i])
            positions_train.append(positions_plume_aug[i]);
    
    return np.array(images_numpy_train, dtype=np.float32), np.array(positions_train, dtype=np.float32), np.array(images_numpy_validation, dtype=np.float32), np.array(positions_validation, dtype=np.float32)

In [177]:
# Hachem : Prepare data sets

BATCH_SIZE = 8
TRAIN_SET_RATIO = 0.7 # Set to 0.8 when not augmenting data. Reason being that we don't include augmented images of validation set images in training set.

# Seperate train and validation sets for PlumeNoPlume
images_numpy_train, labels_train, images_numpy_validation, labels_validation = prepare_plume_noplume_train_validation(plume_arrays,
                                                                                                    plume_arrays_ids,
                                                                                                    
                                                                                                    no_plume_arrays,
                                                                                                    no_plume_arrays_ids,
                                                                                                    
                                                                                                    plume_arrays_aug_imp,
                                                                                                    plume_arrays_aug_imp_ids,
                                                                                                    
                                                                                                    no_plume_arrays_aug_imp,
                                                                                                    no_plume_arrays_aug_imp_ids,
                                                                                                    
                                                                                                    TRAIN_SET_RATIO)

# Seperate train and validation sets for Position estimation
images_numpy_train_p, positions_train_p, images_numpy_validation_p, positions_validation_p = prepare_plume_positions_train_validation(plume_arrays,
                                                                                                                    plume_arrays_ids,
                                                                                                                    plume_positions,
                                                                                                                    
                                                                                                                    plume_arrays_aug_imp,
                                                                                                                    plume_arrays_aug_imp_ids,
                                                                                                                    plume_positions_aug_imp,
                                                                                                                    
                                                                                                                    TRAIN_SET_RATIO)

# Use Appropriate DataLoaders for PlumeNoPlume
train_set_plumenoplume = PlumeNoPlumeDataSet(images_numpy_train, labels_train, transform = transform);
validation_set_plumenoplume = PlumeNoPlumeDataSet(images_numpy_validation, labels_validation, transform = transform);

# Use Appropriate DataLoaders for PlumePositions
train_set_positions = PlumePositionDataSet(images_numpy_train_p, positions_train_p, transform = transform);
validation_set_positions = PlumePositionDataSet(images_numpy_validation_p, positions_validation_p, transform = transform);

# Create DataLoaders for PlumeNoPlume
trainloader_plumenoplume = torch.utils.data.DataLoader(train_set_plumenoplume, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
validationloader_plumenoplume = torch.utils.data.DataLoader(validation_set_plumenoplume, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# Create DataLoaders for PlumePositions
trainloader_positions = torch.utils.data.DataLoader(train_set_positions, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
validationloader_positions = torch.utils.data.DataLoader(validation_set_positions, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

classes = ('no_plume', 'plume')

In [178]:
# Start with CNN classes

class Net(nn.Module):
    """ Classifier model class """
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding_mode='zeros', device=DEVICE)  # Increased filters
        self.bn1 = nn.BatchNorm2d(16, device=DEVICE)  # Batch normalization
        self.pool = nn.MaxPool2d(2, 2)
        self.dp1 = nn.Dropout2d(0.25)  # Dropout

        self.conv2 = nn.Conv2d(16, 32, 5, device=DEVICE)  # Increased filters
        self.bn2 = nn.BatchNorm2d(32, device=DEVICE)  # Batch normalization

        self.fc1 = nn.Linear(5408, 120, device=DEVICE)  # Adjusted input size
        self.dp2 = nn.Dropout(0.5)  # Dropout after fully connected
        self.fc2 = nn.Linear(120, 48, device=DEVICE)
        self.fc3 = nn.Linear(48, 5, device=DEVICE)
        self.fc4 = nn.Linear(5, 1, device=DEVICE)

    def _up_to_features(self, x):
        # Conv1 + BatchNorm + Activation
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.dp1(x)

        # Conv2 + BatchNorm + Activation
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.dp1(x)

        # Flatten
        x = torch.flatten(x, 1)

        # Linear links in between + Dropout + Activation
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp2(x)

        x = self.fc2(x)
        x = F.relu(x)

        x = self.fc3(x)
        return x

    def forward(self, x):
        x = self._up_to_features(x)

        x = self.fc4(x)
        x = F.sigmoid(x)

        return x

class NetPosition(nn.Module):
    """ Position Prediction model class """
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding_mode='zeros', device=DEVICE)  # Increased filters
        self.bn1 = nn.BatchNorm2d(16, device=DEVICE)  # Batch normalization
        self.pool = nn.MaxPool2d(2, 2)
        self.dp1 = nn.Dropout2d(0.25)  # Dropout

        self.conv2 = nn.Conv2d(16, 32, 5, device=DEVICE)  # Increased filters
        self.bn2 = nn.BatchNorm2d(32, device=DEVICE)  # Batch normalization

        self.fc1 = nn.Linear(5408, 120, device=DEVICE)  # Adjusted input size
        self.dp2 = nn.Dropout(0.5)  # Dropout after fully connected
        self.fc2 = nn.Linear(120, 48, device=DEVICE)
        self.fc3 = nn.Linear(48, 5, device=DEVICE)
        self.fc4 = nn.Linear(5, 2, device=DEVICE) # x and y

    def _up_to_features(self, x):
        # Conv1 + BatchNorm + Activation
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.dp1(x)

        # Conv2 + BatchNorm + Activation
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.dp1(x)
        
        # Flatten
        x = torch.flatten(x, 1)

        # Linear links in between + Dropout + Activation
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp2(x)

        x = self.fc2(x)
        x = F.relu(x)

        x = self.fc3(x)
        return x

    def forward(self, x):
        x = self._up_to_features(x)

        x = self.fc4(x)
        x = F.sigmoid(x)

        return x

In [179]:
# [Hachem] Init PlumeNoPlume CNN here
net = Net() 
net = net.to(DEVICE)
print("Model Net Reset")

Model Net Reset


In [180]:
# [Hachem] Init PlumePositions CNN here
net_positions = NetPosition()
net_positions = net_positions.to(DEVICE)
print("Positions model reset")

Positions model reset


In [183]:
# [Hachem] Define functions for calculating truth table [False, True, True, ....] that is further used for validation
def truth_table_plume_noplume(predicted, labels):
    """ 
    Calculates truth table for label prediction
    Inputs :
        - predicted : Values predicted by the model
        - expected : Expected values
    Outputs :
        - Truth table
    """
    return torch.round(predicted).squeeze()==labels.squeeze()

# [Hachem] Same thing but for positions. It is essential to define epsilon as it will consider two points the same if they are epsilon apart.
def truth_table_position(predicted, expected, epsilon = 1e-4):
    """
    Calculates truth table for position prediction
    Epsilon gives the verification radius
    Inputs :
        - predicted : Values predicted by the model
        - expected : Expected values
        - epsilon : Tolerated distance between two points
    Outputs :
        - Truth table
    """
    return (torch.pow(predicted - expected, 2).sum(1).sqrt() < epsilon)

def evaluate_accuracy(model, dataloader, truth_table_function):
    """
    Inputs : 
        - model : model to evaluate
        - dataloader : DataLoader on which to run the evaluation
        - truth_table_function : Truth function used to calculate accuracy
    Outputs :
        - Accuracy on given set.
    """
    # Set to evaluation mode
    model.eval()
    
    # [Hachem] Train accuracy
    correct = 0
    total = 0
    with torch.no_grad():
        
        for i, data in enumerate(dataloader):
            # [Hachem] Extract data and put in GPU (eventually)
            images, results = data
            images, results = images.to(DEVICE), results.to(DEVICE)

            # Get outputs
            predicted = model(images)

            # Increment accordingly total and correct
            total += results.size(dim=0)
            correct += truth_table_function(predicted, results).sum().item()
        
        print(f'Accuracy of the model on train images: {100 * correct / total}%')

    # Return accuracy
    return correct/total

In [184]:
# [Hachem] Define loss function and optimization algorithm
PRINT_EPOCHS_FREQUENCY = 10

def train_model_one_epoch(model, trainloader, loss_criterion, optimizer ):
    """
    Trains model over one epoch
    Inputs :
        - model : model to train over one epoch
        - trainloader : DataLoader containing the training DataSet
        - loss_criterion : Loss function to use
        - optimizer : optimizer in use for training
    Ouputs :
        - running loss at the end of training
    """
    criterion = loss_criterion
    optimizer = optimizer
    
    running_loss = 0.0
    counter = 0
    for i, data in enumerate(trainloader):
        counter += 1
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        # [hachem] Set training mode
        model.train()

        # labels = labels.long();
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)

        if outputs.size(0) > 1:
            outputs = outputs.squeeze()
        else:
            outputs = outputs.view(-1)

        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    
    return running_loss

def train_net(model, trainloader, validationloader, numofepochs = 100, save_when_accuracy_higher_than = 0.82, save_multiplier = 1.001):
    """
    Trains Classifier Network
    Inputs :
        - model : Model to train
        - trainloader : DataLoader for training
        - validationloader : DataLoader for validation
        - numofepochs : Number of epochs of training
        - save_when_accuracy_higher_than : Save file if accuracy goes beyond this value
        - save_miltiplier : Multiplication coefficient for further saves.
    Outputs :
        None.
    """
    criterion = nn.SmoothL1Loss()
    optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9, weight_decay=0.001)
    
    # [Hachem] Train    
    print("Size of training set : ~ ", len(trainloader) * BATCH_SIZE)
    print("Size of validation set : ~ ", len(validationloader) * BATCH_SIZE)
    
    for epoch in range(numofepochs):  # loop over the dataset multiple times
        # Run training for this epoch.
        running_loss = train_model_one_epoch(model, trainloader, criterion, optimizer)
        # Depending on PRINT_EPOCHS_FREQUENCY :
        if epoch % PRINT_EPOCHS_FREQUENCY == 0:
            print(f'[{epoch + 1}] loss: {running_loss / BATCH_SIZE:.3f}')
            # Print train accuracy
            evaluate_accuracy(model, trainloader, truth_table_plume_noplume) 
            # Print validation accuracy
            validation_accuracy = evaluate_accuracy(model, validationloader, truth_table_plume_noplume)
            if validation_accuracy > save_when_accuracy_higher_than:
                print("Saving model");
                torch.save(net.state_dict(), PREPENDING_PATH + "best_classifier_model.pt")
                max_result = correct/total * save_multiplier

def train_net_position(model, trainloader, validationloader, numofepochs = 100, epsilon = 0.1, save_when_accuracy_higher_than = 0.82, save_multiplier = 1.001):
    """
    Trains Position Predicting Network
    Inputs :
        - model : Model to train
        - trainloader : DataLoader for training
        - validationloader : DataLoader for validation
        - numofepochs : Number of epochs of training
        - epsilon : Tolerated distance for validation
        - save_when_accuracy_higher_than : Save file if accuracy goes beyond this value
        - save_miltiplier : Multiplication coefficient for further saves.
    Outputs :
        None.
    """
    criterion = nn.HingeEmbeddingLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    print("Size of training set : ~ ", len(trainloader) * BATCH_SIZE)
    print("Size of validation set : ~ ", len(validationloader) * BATCH_SIZE)
    
    for epoch in range(numofepochs):  # loop over the dataset multiple times
        # Run training for this epoch.
        running_loss = train_model_one_epoch(model, trainloader, criterion, optimizer)
        # Depending on PRINT_EPOCHS_FREQUENCY :
        if epoch % PRINT_EPOCHS_FREQUENCY == 0:
            print(f'[{epoch + 1}] loss: {running_loss / BATCH_SIZE:.3f}')
            # Print train accuracy
            evaluate_accuracy(model, trainloader, truth_table_plume_noplume) 
            # Print validation accuracy
            validation_accuracy = evaluate_accuracy(model, validationloader, truth_table_position)
            if validation_accuracy > save_when_accuracy_higher_than:
                print("Saving model");
                torch.save(net.state_dict(), PREPENDING_PATH + "best_positions_model.pt")
                max_result = correct/total * save_multiplier    

# 5. Start Training

In [192]:
# Train Net network
if True:
    print("Now training Classifier CNN")
    train_net(net, trainloader_plumenoplume, validationloader_plumenoplume, 400)
    print('Finished Training CNN Net')

Now training Classifier CNN
Size of training set : ~  3104
Size of validation set : ~  120
[1] loss: 3.980
Accuracy of the model on train images: 80.83870967741936%
Accuracy of the model on train images: 77.5%
[11] loss: 3.882
Accuracy of the model on train images: 81.96774193548387%
Accuracy of the model on train images: 78.33333333333333%
[21] loss: 3.748
Accuracy of the model on train images: 84.87096774193549%
Accuracy of the model on train images: 80.0%
[31] loss: 3.627
Accuracy of the model on train images: 84.83870967741936%
Accuracy of the model on train images: 76.66666666666667%
[41] loss: 3.455
Accuracy of the model on train images: 86.38709677419355%
Accuracy of the model on train images: 77.5%
[51] loss: 3.360
Accuracy of the model on train images: 87.48387096774194%
Accuracy of the model on train images: 77.5%
[61] loss: 3.279
Accuracy of the model on train images: 86.83870967741936%
Accuracy of the model on train images: 75.0%
[71] loss: 3.004
Accuracy of the model on tr

In [187]:
# Train Positions Network
if True:
    print("Now training Position CNN")
    number_of_epochs_for_position_training = 10
    epsilon_distance = 0.2
    train_net_position(net_positions, trainloader_positions, validationloader_positions, number_of_epochs_for_position_training, epsilon_distance)
    print("Now Position CNN finished")

Now training Position CNN
Size of training set : ~  1568
Size of validation set : ~  64
[1] loss: 24.227
Accuracy of the model on train images: 1.9218449711723253%
Accuracy of the model on train images: 0.0%
Now Position CNN finished


In [189]:
# Note : the following code was not as successful.
# [Hachem] Train XGBoost with a well-chosen (hopefully) hidden layer from CNN.
features_train_xgboost = []
labels_train_xgboost = []

features_validation_xgboost = []
labels_validation_xgboost = []


with torch.no_grad():
    for data, target in trainloader_plumenoplume:
        data = data.to(device)
        output = F.relu(net._up_to_features(data).to('cpu'))
        features_train_xgboost.extend(output.numpy())
        labels_train_xgboost.extend(target.numpy())

    for data, target in validationloader_plumenoplume:
        data = data.to(device)
        output = F.relu(net._up_to_features(data).to('cpu'))
        features_validation_xgboost.extend(output.numpy())
        labels_validation_xgboost.extend(target.numpy())

# Convert to numpy arrays
features_train_xgboost = np.array(features_train_xgboost)
features_validation_xgboost = np.array(features_validation_xgboost)
labels_train_xgboost = np.array(labels_train_xgboost)
labels_validation_xgboost = np.array(labels_validation_xgboost)

In [190]:
import xgboost as xgb

# Convert to DMatrix
dtrain = xgb.DMatrix(features_train_xgboost, label=labels_train_xgboost)

# Set parameters and train
params = {
    'max_depth': 2,
    'eta': 0.01,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'
}
num_round = 100
bst = xgb.train(params, dtrain, num_round)


In [191]:
# [Hachem] CNN + XGBoost Train accuracy
predict_train_xgboost = bst.predict(dtrain)
predict_train_xgboost = np.round(predict_train_xgboost)

print(predict_train_xgboost)
accuracy = np.sum(predict_train_xgboost == labels_train_xgboost) / len(labels_train_xgboost)
print(f'Accuracy of the model + XGBoost on train images: {100 * accuracy:.2f}%')


# [Hachem] CNN + XGBoost validation accuracy
dvalidation = xgb.DMatrix(features_validation_xgboost)

predict_validation_xgboost = bst.predict(dvalidation)
predict_validation_xgboost = np.round(predict_validation_xgboost)

accuracy = np.sum(predict_validation_xgboost == labels_validation_xgboost) / len(labels_validation_xgboost)
print(f'Accuracy of the model + XGBoost on validation images: {100 * accuracy:.2f}%')


[1. 0. 0. ... 0. 1. 1.]
Accuracy of the model + XGBoost on train images: 76.74%
Accuracy of the model + XGBoost on validation images: 60.00%
